In [104]:
import pandas as pd
import numpy as np

In [105]:
mapping = pd.read_csv('C:/Users/ScMa9004/Downloads/AREA_CELL_MAPPING_1579159674081.csv', sep = ',', low_memory=False)

In [106]:
mapping = mapping[['INDEX_ID', 'MICRO_CELL_RULE_NAME', 'MICRO_CELL_RULE_ID', 'CELL_AREA_RULE_ID']].drop_duplicates()

In [107]:
df = pd.read_csv('C:/Users/ScMa9004/Downloads/RULES_1579101873184.csv', sep = ',', low_memory = False)

In [108]:
# из выгрузки правил вырезать нужные
df = df[df['CREATED_BY'] == 'maxim.schenin@nielsen.com']

In [110]:
cells = pd.read_excel('C:/Users/ScMa9004/Scripts/new_panel_op_ap_314.xlsx')

In [111]:
cells = cells[['CELL_ID', 'new_mcell_name', 'AP', 'OP']]

In [112]:
cells = cells.rename(columns = {'new_mcell_name': 'MICRO_CELL_RULE_NAME'})

In [113]:
res = pd.DataFrame(columns = {'CELL_STORE'})

# CELL_STORE MAPPING

In [114]:
for i in df['RULE_ID'][df['RULE_TYPE'] == 'MICRO_CELL'].unique():
    try:
        a = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_STORE') &
            (df['ATTRIBUTE_ID'] == 6) &
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 6)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 6)].iloc[0])]
    except:
        a = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_STORE'].unique()
    try:
        b = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_STORE') &
                    (df['ATTRIBUTE_ID'] == 24) &      
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 24)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 24)].iloc[0])]
    except:
        b = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_STORE'].unique()
    try:
        c = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_STORE') &
                        (df['ATTRIBUTE_ID'] == 15) &  
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 15)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 15)].iloc[0])]
    except:
        c = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_STORE'].unique()
    try:
        d = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_STORE') &
                          (df['ATTRIBUTE_ID'] == 9227) &
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 9227)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 9227)].iloc[0])]
    except:
        d = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_STORE'].unique()
        
    e = set(a) & set(b) & set(c) & set(d)
    
    res.loc[i, 'CELL_STORE'] = e
    
res = res.reset_index(drop = False)
    
res['CELL_STORE'] = res['CELL_STORE'].astype('str')
    
res['CELL_STORE'] = res['CELL_STORE'].str.replace('{', '')
res['CELL_STORE'] = res['CELL_STORE'].str.replace('}', '')

for i, j in res.iterrows():
    if len(j['CELL_STORE'].split(',')) > 1:
        if len(df[df['RULE_ID'] == int(j['CELL_STORE'].split(',')[0])]) ==\
        len(df[(df['RULE_ID'] == j['index']) &\
                (df['ATTRIBUTE_NAME'].isin(df['ATTRIBUTE_NAME'][df['RULE_ID'] == int(j['CELL_STORE'].split(',')[0])]))]):
            res.loc[i, 'CELL_STORE'] = int(j['CELL_STORE'].split(',')[0])
        else:
            res.loc[i, 'CELL_STORE'] = int(j['CELL_STORE'].split(',')[1])
            
res = res.astype('int')

In [115]:
res = res.rename(columns = {'index': 'MICRO_CELL_RULE_ID'})

In [116]:
res = pd.merge(res, mapping, on = 'MICRO_CELL_RULE_ID', how = 'inner')

In [117]:
res = pd.merge(res, cells, on = 'MICRO_CELL_RULE_NAME', how = 'inner')

In [118]:
res = res.rename(columns = {'INDEX_ID': 'INDEX',
                            'CELL_ID': 'CELL',
                            'MICRO_CELL_RULE_ID': 'MICRO_CELL_ID', 
                            'OP': 'TARGET_SAMPLE_SIZE', 
                            'AP': 'MAX_SAMPLE_SIZE', 
                            'CELL_STORE': 'CELL_STORE_RULE_ID', 
                            })

In [119]:
res.loc[res['INDEX'] != 1001395, 'PERIOD'] = 'MONT_1101'
res.loc[res['INDEX'] == 1001395, 'PERIOD'] = 'RMEX_1101'
res['IDEAL_SAMPLE_SIZE'] = 0
res['OWNERSHIP'] = 'IO'
res['CENSUS'] = 'N'
res['IS_ACTIVE'] = 'Y'

In [120]:
res = res[['PERIOD', 'INDEX', 'CELL', 'MICRO_CELL_ID', 'IDEAL_SAMPLE_SIZE', 'TARGET_SAMPLE_SIZE', 
           'MAX_SAMPLE_SIZE', 'CELL_STORE_RULE_ID', 'CELL_AREA_RULE_ID', 'OWNERSHIP', 'CENSUS', 'IS_ACTIVE']]

In [121]:
res_fmcg = res[res['INDEX'] == 1000296]
res_cig = res[res['INDEX'] == 1000334]
res_beer = res[res['INDEX'] == 1000328]
res_exxon = res[res['INDEX'] == 1001395]

In [122]:
res_fmcg.to_csv('Pandel_design_fmcg.csv', index = False, sep = ',', encoding = 'utf8')
res_cig.to_csv('Pandel_design_cig.csv', index = False, sep = ',', encoding = 'utf8')
res_beer.to_csv('Pandel_design_beer.csv', index = False, sep = ',', encoding = 'utf8')
res_exxon.to_csv('Pandel_design_exxon.csv', index = False, sep = ',', encoding = 'utf8')